## Initialize client

In [1]:
import fiddler as fdl
client = fdl.FiddlerApi()

project_id = 'tutorial'
model_id = 'wine_quality_model'

## Load event log

In [3]:
import pandas as pd
event_log = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/events.log')


## Publish events

In [4]:
import datetime
import random
from IPython.display import clear_output
import time

two_weeks_ms = 604800 * 2 * 1000

chunked_df = [event_log[i:i+1] for i in range(0, event_log.shape[0], 1)]
count = 0
for row_df in chunked_df:
    data = row_df.to_dict('records')[0]
    occurred_at = int(round(time.time() * 1000)) - random.randint(0, two_weeks_ms)

    result = client.publish_event(project_id, model_id, data, event_time_stamp=occurred_at)
    
    count += 1
    clear_output(wait = True)
    print(f'Sending {count} / {len(event_log)}')
    time.sleep(2)

Sending 3 / 1139


KeyboardInterrupt: 